In [3]:
# PATTERN MATCHING
%matplotlib inline
from pylab import *
import matplotlib.mlab
import wave
import numpy as np
from scipy.io import wavfile
from scipy import signal
from scipy import interpolate
import numpy.matlib
import sqlite3 as lite
from PyQt4 import QtCore, QtGui
import winsound

DB = 'E:\Courses\SecondSem\MIR\projectmaterial\DB\QBH_MIR.db'
path='E:\Courses\SecondSem\MIR\projectmaterial\DB';
connection = lite.connect(DB)

def findSongs():
    
    #DB = 'E:\Courses\SecondSem\MIR\projectmaterial\DB\QBH_MIR.db'
    #path='E:\Courses\SecondSem\MIR\projectmaterial\DB';
    # Connection to DB
    #connection = lite.connect(DB)
    
    
    def calcDbSongsDistances(dbSongs, str2):
        distances=[]
        for i in range(0,len(dbSongs)):
            distance=dynamicEditDistance(dbSongs[i],str2)
            print (distance)
            distances.append(distance)
        return distances;
        
    
    def dynamicEditDistance(str1, str2):
        temp= np.zeros((len(str1)+1, len(str2)+1)); #creating a temp of rows of length as of str1 and cols of str2
        for i in range(0, len(temp[0])):
            temp[0][i] = i
        for i in range(0, len(temp)):
            temp[i][0] = i
        for i in range(1, len(str1)+1):
            for j in range(1, len(str2)+1):
                if str1[i-1]==str2[j-1]:
                    temp[i][j] = temp[i-1][j-1]
                else:
                    temp[i][j] = 1 + min(temp[i-1][j-1], temp[i-1][j], temp[i][j-1])
        return temp[len(str1)][len(str2)];
        
    def fetchDBSongs():
        with connection:
            
            cursor = connection.cursor() 
            connection.row_factory = lambda cursor, row: row[0]
            c = connection.cursor()
            patterns = c.execute('SELECT pattern FROM audioFeature').fetchall()
            #print ("Dataset Patterns: ",patterns)
            connection.commit()   
        return patterns;
        
    def fetchHummedSong():
    
        with connection:
            
            cursor = connection.cursor() 
            connection.row_factory = lambda cursor, row: row[0]
            c = connection.cursor()
            pattern = c.execute('SELECT pattern FROM hummedFeature').fetchone()
            #print ("Hummed Pattern: ",pattern)
            fname = c.execute('SELECT queryTone FROM hummedFeature').fetchone()
            print (fname)
            #winsound.PlaySound(fname, winsound.SND_FILENAME)
            connection.commit()
        return pattern;
       
    def updateDistance():
        #print ("insertInDB ");
        #Insert values in database
        DB = path+'\QBH_MIR.db'
        #print (len(dbSongs));
    
        connection = lite.connect(DB)
        for i in range(len(dbSongs)):
            with connection:
                cursor = connection.cursor()
                sql = "UPDATE audioFeature SET distance=? WHERE pattern=?";
                connection.execute(sql,[result[i], dbSongs[i]]) 
                connection.commit()
        
    def fetchRankedSongs():
        with connection:
            cursor = connection.cursor() 
            connection.row_factory = lambda cursor, row: row[0]
            c = connection.cursor()
            names = c.execute('SELECT songName FROM audioFeature ORDER BY distance ASC LIMIT 5').fetchall()
            distances = c.execute('SELECT distance FROM audioFeature ORDER BY distance ASC LIMIT 5').fetchall()
            ids = c.execute('SELECT id FROM audioFeature ORDER BY distance ASC LIMIT 5').fetchall()
            songPath = c.execute('SELECT song FROM audioFeature ORDER BY distance ASC LIMIT 5').fetchall()
            imagePath = c.execute('SELECT ImageName FROM audioFeature ORDER BY distance ASC LIMIT 5').fetchall()
            #print ("Dataset distance: ",distance)
            connection.commit()   
        return names,distances,ids,songPath,imagePath;
    
    hummedSong = fetchHummedSong();
    dbSongs=fetchDBSongs(); # Collection of songs
    result=calcDbSongsDistances(dbSongs,hummedSong); #Creating list of min distances of all songs in DB from hummed Query
    #print (result);
    updateDistance();
    songs=[];
    allList=[];
    RankedSongs, distances, ids,songPath,imagePath = fetchRankedSongs();
    for i in range(len(distances)):
        songs=ids[i], RankedSongs[i], distances[i],songPath[i],imagePath[i] ;
        #print (songs);
        allList.append(songs);
    print (allList);
    
    with connection:    
        for i in range (0,len(allList)):
            cursor = connection.cursor() 
            sql1 = "UPDATE matchedSongs SET songName=?,distance=?,queryTone=?,ImageName=? WHERE id=?";
            cursor.execute(sql1, [RankedSongs[i],distances[i],songPath[i],imagePath[i],i+1])
            connection.commit()
    connection.close()

    
#findSongs();
connection.close()

In [20]:
import os
def showContour():
        path="E:\Courses\SecondSem\MIR\projectmaterial\DB\Dataset\Audio2.wav"
        print (path)
        print (path.rsplit('\\', 1)[1])
            
showContour()

E:\Courses\SecondSem\MIR\projectmaterial\DB\Dataset\Audio2.wav
Audio2.wav


In [ ]:
#show contour
#For hummed query

import sqlite3 as lite
import os.path
from os import listdir, getcwd
from IPython.display import display, Image

DB = 'E:\Courses\SecondSem\MIR\projectmaterial\DB\QBH_MIR.db'

connection = lite.connect(DB)

id=1; #always 1

def fetchHummedContour():
    with connection:
        
        cursor = connection.cursor() 
        connection.row_factory = lambda cursor, row: row[0]
        c = connection.cursor()
        path = c.execute("SELECT ImageName FROM hummedFeature WHERE id=?", (id,))
        connection.commit()
    return path;

def picMain():
    path=fetchHummedContour();
    y = [ip for ip in path]
    newPath=','.join(y);
    img = Image(newPath)
    display(img)

#picMain();

In [ ]:
#show contour
#For Dataset

import sqlite3 as lite
import os.path
from os import listdir, getcwd
from IPython.display import display, Image

DB = 'E:\Courses\SecondSem\MIR\projectmaterial\DB\QBH_MIR.db'

connection = lite.connect(DB)

id=1; # change to path

def fetchSongContour():
    with connection:
        
        cursor = connection.cursor() 
        connection.row_factory = lambda cursor, row: row[0]
        c = connection.cursor()
        path = c.execute("SELECT ImageName FROM audioFeature WHERE id=?", (id,))
        connection.commit()
    return path;

def picMain():
    path=fetchSongContour();
    y = [ip for ip in path]
    newPath=','.join(y);
    img = Image(newPath)
    display(img)

#picMain();